# PS1

## 1. Flowchart

In [3]:
def Print_values(a, b, c):
    if a > b:
        if b > c:
            lst = [a, b, c]
        else:
            if a > c:
                lst = [a, c, b]
            else:
                lst = [c, a, b]
    else:
        if b > c:
            if a > c:
                lst = [a, c, b]
            else:
                lst = [c, a, b]
        else:
            lst = [c, b, a]
    output = lst[0]+lst[1]-10*lst[2]
    print(output)

In [7]:
import random

a = round(random.random()*100)
b = round(random.random()*100)
c = round(random.random()*100)
print(a,b,c)

Print_values(a, b, c)

96 82 78
-602


In [8]:
a = 10
b = 5
c = 1

Print_values(a, b, c)

5


## 2. Continuous ceiling function

In [37]:
import math
import random

# Generate a list with N random positive integers
N = 5
inputs = [round(random.random()*100) for x in range(N)]
inputs

[93, 45, 90, 23, 75]

In [38]:
# Define a function to find the ceiling of x/3
def find_ceil(x):
    ceil = math.ceil(x/3)
    return ceil

In [39]:
# The main F(x) function using a while loop
# continually calling the find_ceil function to reduce x to integer 1 
def F(x):
    
    F_1 = 1
    add_all=0

    while x/3 > 1/3:
        double_x.append(2*x)
        add_all = add_all+2*x
        x = find_ceil(x)
        find_ceil(x)
        
    output = F_1+add_all
    return output

In [40]:
# Find and print all the outputs in a list
outputs = []

for x in inputs:
    outputs.append(F(x))
    
print(outputs)

[283, 135, 273, 69, 225]


## 3. Dice rolling

### 3.1

### 3.2

## 4. Dynamic programming

### 4.1

### 4.2

### 4.3

## 5. Path counting

### 5.1

### 5.2

### 5.3